## Figure 6
Results from a parcel simulation with Na = 100 cm-3, Forg = 0.5, and w = 5 mm/s.
Difference in CDNC profiles between the different surface-partitioning models and constant surface tension.
Solid lines show best fit parameter values, while shading indicates the variability from changing the parameter values within 1std.

Comparing the structural vs. parametric uncertainty. Are shadings wider than difference between curves?